In [ ]:
# !py -3 -m pip install --upgrade Jinja2==3.0.3
# !py -3 -m pip install pandas
# !py -3 -m pip install teradatasql
# !py -3 -m pip install teradatasqlalchemy

In [ ]:
import pandas as pd
import sqlalchemy
import os
import json
from jinjasql import JinjaSql

In [ ]:
with open('../config/td_config.json') as json_td_config:
    config = json.load(json_td_config)

USER_NAME = config['user_name']
PASSWORD = config['pword']
HOST_NAME = config['host_name']

In [ ]:
td_engine = sqlalchemy.create_engine('teradatasql://' + HOST_NAME + '/?user=' + USER_NAME + '&password=' + PASSWORD + '&logmech=KRB5')

In [ ]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\mirror_tables_check.csv"
output_folder = "OutputFiles\\"
output_file = 'mirrored_results.csv'
output_path = f"{output_folder}{output_file}"

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [ ]:
#Run Mirror Table Check 
def run_mirror_table_check():
    try:
        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        
        file_df = pd.read_csv(input_path, index_col=None)
        file_df.sort_values(by=['Database_Name','Table_Name'],inplace=True)
        datasets = ','.join(f"'{database}'" for database in file_df.Database_Name.unique())

        query = "SELECT databasename as Database_Name, tablename as Table_Name, wherecond as Criteria, refreshappendcdc as Mirroring_Type FROM edw_pub_views.mirroring_status WHERE databasename IN ({});".format(datasets)
        table_df = pd.read_sql(query, td_engine)
        table_df.sort_values(by=['Database_Name','Table_Name'],inplace=True)

        combined_df = file_df.merge(table_df, how='outer', on=['Database_Name','Table_Name'], sort=True, indicator=True, validate="one_to_one")

        mirrored_status_list = []
        mirrored_status_list.append("Dataset,Table,Status,Criteria,Mirroring_Type")
        # print(combined_df)
        for idx in combined_df.index:
            database = combined_df['Database_Name'][idx]
            table = combined_df['Table_Name'][idx]
            
            match combined_df['_merge'][idx]:
                case 'both':
                    status = 'Mirrored'
                    criteria = combined_df['Criteria'][idx]
                    mirroring_type = combined_df['Mirroring_Type'][idx]                    
                case 'right_only':
                    status = 'Not Found in Analysis'
                    criteria = combined_df['Criteria'][idx]
                    mirroring_type = combined_df['Mirroring_Type'][idx]
                case 'left_only':
                    status = 'Not Mirrored'
                    criteria = ""
                    mirroring_type = ""

            mirrored_status_list.append(f"{database},{table},{status},{criteria},{mirroring_type}")
        
        write_file_local(output_path, mirrored_status_list)

    except Exception as e1:
        print(e1)
        pass

In [ ]:
print("Begin of Processing")

run_mirror_table_check()

print("End of Processing")